In [ ]:
!pip install faiss-cpu sentence-transformers openai



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 57.3 MB/s eta 0:00:00


In [ ]:
import faiss
import numpy as np
import openai
from sentence_transformers import SentenceTransformer
from openai import OpenAI


open.api_key = "my_key_here"#didnt buy an api key still
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
#initialize faiss index
dimension = 384
index = faiss.IndexFlatL2(dimension)


data = [
    {"id": "1", "text": "Our business hours are 9 AM to 5 PM, Monday to Friday."},
    {"id": "2", "text": "We offer free shipping on orders over 1000."},
    {"id": "3", "text": "For technical support, contact support@example.com."},
    {"id": "4", "text": "You can track your order status using the tracking number sent to your email."},
    {"id": "5", "text": "To reset your password, go to the 'Forgot Password' section on the login page."},
    {"id": "6", "text": "We accept all major credit cards, PayPal, and Apple Pay for payments."},
    {"id": "7", "text": "If you're not satisfied with your purchase, we offer a 30-day return policy."},
    {"id": "8", "text": "Our office is located at PLOT NO A PART BN REDDY COLONY, CHERLAPALLI , HYDERABAD, Telangana."},
]

embeddings = np.array([embedding_model.encode(item["text"]) for item in data])

# Add embeddings to FAISS index
index.add(embeddings)



def retrieve_context(query, top_k=3):
    query_embedding = embedding_model.encode(query).reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)
    return [data[i]["text"] for i in indices[0]]


def generate_answer(query, top_k=3):
    # Retrieve context from FAISS
    context = retrieve_context(query, top_k)


    context_str = "\n".join(context)

    # Generating my answer (used openai here)
    prompt = f"Context:\n{context_str}\n\nQuestion: {query}\nAnswer:"

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=150,
        temperature=0.7
    )

    return response.choices[0].text.strip()

# Test the system with a query
query = "What are your business hours?"
answer = generate_answer(query)
print("Answer:", answer)

